In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [4]:
article_tags = pd.read_csv("articles_tags.csv")
article_text = pd.read_csv("../resources/articles_text.csv")
article_tags= article_tags.set_index("Resource URL")
article_text = article_text.set_index("Resource URL")
article_text = article_text[~article_text.index.duplicated(keep='first')]

In [5]:
for i in article_text.index:
    article_text.loc[i, "tech_tag"] =  article_tags.loc[i]["Technical (Y/N)"][0]

In [6]:
data = (article_text[(article_text["tech_tag"]=="Y") | (article_text["tech_tag"]=="N")])
data_src = data
data_src = data_src[["Resource Title", "text", "tech_tag"]]
data_src.loc[:, "text"] =  data_src["Resource Title"] + " " +data_src["text"]
data_src.loc[:, "tech_tag"] = data_src["tech_tag"].map({"N": 0, "Y":1})
data_src.drop("Resource Title", axis=1, inplace=True)
data_src = data_src.dropna().reset_index(drop=True)
data=data_src

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
import tensorflow as tf

In [8]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [32]:
dataset = tf.data.Dataset.from_tensor_slices((data.text, data.tech_tag))


In [10]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [11]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [12]:
train_text =  dataset.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [13]:
def binary_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return binary_vectorize_layer(text), label

In [14]:
def int_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

In [15]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(dataset))


In [72]:
!pip install tensorflow-text

In [18]:
import tensorflow_text as tf_text

In [19]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [20]:
def tokenize(text, unused_label):
    lower_case = tf_text.case_fold_utf8(text)
    return tokenizer.tokenize(lower_case)

In [34]:
all_labeled_data=dataset

In [35]:
for text, label in all_labeled_data.take(10):
    print("Sentence: ", text.numpy()[:10])
    print("Label:", label.numpy())

Sentence:  b'Efficacy o'
Label: 1
Sentence:  b'Effectiven'
Label: 1
Sentence:  b'Updated Ma'
Label: 1
Sentence:  b'The New En'
Label: 1
Sentence:  b'SARS-CoV-2'
Label: 1
Sentence:  b'Global Inc'
Label: 1
Sentence:  b'Missing th'
Label: 0
Sentence:  b'Mass-Vacci'
Label: 0
Sentence:  b'Antibody R'
Label: 1
Sentence:  b'Coronaviru'
Label: 1


In [36]:
tokenized_ds = all_labeled_data.map(tokenize)

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [39]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)


In [41]:
import collections

In [42]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

Vocab size:  10000
First five vocab entries: [b'the', b',', b'of', b'.', b'and']


In [43]:
keys = vocab
values = range(2, len(vocab) + 2)  # reserve 0 for padding, 1 for OOV

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

In [44]:
def preprocess_text(text, label):
    standardized = tf_text.case_fold_utf8(text)
    tokenized = tokenizer.tokenize(standardized)
    vectorized = vocab_table.lookup(tokenized)
    return vectorized, label

In [47]:
all_encoded_data = all_labeled_data.map(preprocess_text)

In [70]:
VALIDATION_SIZE=200
BUFFER_SIZE = 50000
BATCH_SIZE = 64

In [71]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [72]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

In [74]:
vocab_size += 2

In [75]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

In [86]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [100]:
def create_rnn(vocab_size, num_labels):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, 64, mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_labels)
    ])
    return model

def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
    ])
    return model

In [ ]:
a = next(iter(train_data))

In [101]:
model = create_rnn(vocab_size=vocab_size, num_labels=2)
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
history = model.fit(train_data, validation_data=validation_data, epochs=10)

Epoch 1/10
13/13 [==============================] - 18s 835ms/step - loss: 0.6786 - accuracy: 0.6550 - val_loss: 0.6524 - val_accuracy: 0.8150
Epoch 2/10
13/13 [==============================] - 8s 629ms/step - loss: 0.4977 - accuracy: 0.8512 - val_loss: 0.5090 - val_accuracy: 0.7500
Epoch 3/10
13/13 [==============================] - 8s 631ms/step - loss: 0.1896 - accuracy: 0.9375 - val_loss: 0.5623 - val_accuracy: 0.7900
Epoch 4/10
13/13 [==============================] - 8s 639ms/step - loss: 0.0904 - accuracy: 0.9737 - val_loss: 0.5032 - val_accuracy: 0.8000
Epoch 5/10
13/13 [==============================] - 8s 643ms/step - loss: 0.0448 - accuracy: 0.9875 - val_loss: 0.5710 - val_accuracy: 0.7850
Epoch 6/10
13/13 [==============================] - 8s 633ms/step - loss: 0.0224 - accuracy: 0.9937 - val_loss: 0.7186 - val_accuracy: 0.7800
Epoch 7/10
13/13 [==============================] - 8s 636ms/step - loss: 0.0092 - accuracy: 0.9987 - val_loss: 0.6531 - val_accuracy: 0.7900
Epoch

In [ ]:
for i in articles

In [93]:
history = model.fit(train_data, validation_data=validation_data, epochs=10)

Epoch 1/10
13/13 [==============================] - 5s 387ms/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.5819 - val_accuracy: 0.7500
Epoch 2/10
13/13 [==============================] - 5s 397ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.5966 - val_accuracy: 0.7500
Epoch 3/10
13/13 [==============================] - 5s 390ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.6095 - val_accuracy: 0.7500
Epoch 4/10
13/13 [==============================] - 5s 409ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.6220 - val_accuracy: 0.7500
Epoch 5/10
13/13 [==============================] - 5s 397ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.6344 - val_accuracy: 0.7500
Epoch 6/10
13/13 [==============================] - 5s 397ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.6450 - val_accuracy: 0.7500
Epoch 7/10
13/13 [==============================] - 5s 419ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.6551 - val_accuracy: 0.7500
Epoch 

In [82]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

4/4 [==============================] - 1s 27ms/step - loss: 0.5100 - accuracy: 0.8000
Loss:  0.5099855661392212
Accuracy: 80.00%
